### 1. Здійсніть імпорт необхідних пакетів.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from category_encoders import TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns



### 2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv

In [2]:
# Завантаження тренувального набору
train_data = pd.read_csv("mod_04_hw_train_data.csv")

# Завантаження валідаційного набору
valid_data = pd.read_csv("mod_04_hw_valid_data.csv")

# Виведемо перші рядки для огляду даних
train_data.head(), valid_data.head()


(                 Name  Phone_Number  Experience  ... Cert Date_Of_Birth  Salary
 0  Jennifer Hernandez  120-602-1220         3.0  ...  Yes    25/08/1972   98000
 1      Timothy Walker  840-675-8650         5.0  ...  Yes    03/12/2013  135500
 2         David Duran  556-293-8643         5.0  ...  Yes    19/07/2002  123500
 3       Gloria Ortega  463-559-7474         3.0  ...   No    19/02/1970   85000
 4      Matthew Steele  968-091-7683         5.0  ...  Yes    20/02/1970  111500
 
 [5 rows x 9 columns],
               Name  Phone_Number  Experience  ... Cert Date_Of_Birth  Salary
 0   Alvaro Johnson  320-636-8883           7  ...   No    12/03/1978  109300
 1    Austin Powers  903-121-1691           2  ...  Yes    13/03/1992   84800
 2      Joshua Phil  673-972-2453           3  ...  Yes    19/02/1988   98900
 3  Mirinda Collins  310-364-6925           5  ...   No    20/03/1989  116500
 4   Mustapha Green  401-249-3912           3  ...  Yes    21/03/1979   75800
 
 [5 rows x 9 column

### 3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

In [3]:
# Перевіримо наявність пропусків у тренувальному та валідаційному наборах даних
missing_train = train_data.isnull().sum()
missing_valid = valid_data.isnull().sum()

In [4]:
missing_data_analysis = pd.DataFrame({
    "Train Missing Before": missing_train,
    "Train Missing After": train_data.isnull().sum(),
    "Valid Missing Before": missing_valid,
    "Valid Missing After": valid_data.isnull().sum()
})

# Виведемо результати аналізу пропусків у текстовому форматі
print("Аналіз пропусків у тренувальному наборі даних:")
print(missing_data_analysis[["Train Missing Before", "Train Missing After"]])

print("\nАналіз пропусків у валідаційному наборі даних:")
print(missing_data_analysis[["Valid Missing Before", "Valid Missing After"]])

Аналіз пропусків у тренувальному наборі даних:
               Train Missing Before  Train Missing After
Name                              0                    0
Phone_Number                      0                    0
Experience                        2                    2
Qualification                     1                    1
University                        0                    0
Role                              3                    3
Cert                              2                    2
Date_Of_Birth                     0                    0
Salary                            0                    0

Аналіз пропусків у валідаційному наборі даних:
               Valid Missing Before  Valid Missing After
Name                              0                    0
Phone_Number                      0                    0
Experience                        0                    0
Qualification                     0                    0
University                        0               

In [5]:
from datetime import datetime

# Видалимо стовпці 'Name' та 'Phone_Number'
train_data.drop(columns=['Name', 'Phone_Number'], inplace=True)
valid_data.drop(columns=['Name', 'Phone_Number'], inplace=True)

# Перетворимо 'Date_Of_Birth' у вік (кількість років)
current_year = datetime.now().year

train_data['Age'] = train_data['Date_Of_Birth'].apply(lambda x: current_year - int(x.split('/')[-1]))
valid_data['Age'] = valid_data['Date_Of_Birth'].apply(lambda x: current_year - int(x.split('/')[-1]))

# Видалимо стовпець 'Date_Of_Birth', оскільки ми вже отримали вік
train_data.drop(columns=['Date_Of_Birth'], inplace=True)
valid_data.drop(columns=['Date_Of_Birth'], inplace=True)

# Перевіримо, як тепер виглядають набори даних
train_data.head(), valid_data.head()


(   Experience Qualification University    Role Cert  Salary  Age
 0         3.0           Msc      Tier2     Mid  Yes   98000   52
 1         5.0           PhD      Tier2  Senior  Yes  135500   11
 2         5.0           Msc      Tier2  Senior  Yes  123500   22
 3         3.0           Bsc      Tier3     Mid   No   85000   54
 4         5.0           Bsc      Tier2  Senior  Yes  111500   54,
    Experience Qualification University    Role Cert  Salary  Age
 0           7           Bsc      Tier1  Senior   No  109300   46
 1           2           Msc      Tier1     Mid  Yes   84800   32
 2           3           Bsc      Tier3     Mid  Yes   98900   36
 3           5           Msc      Tier2  Senior   No  116500   35
 4           3           PhD      Tier1  Junior  Yes   75800   45)

In [6]:
# Заповнимо пропуски у тренувальному наборі
train_data.fillna({
    'Experience': train_data['Experience'].mean(),
    'Qualification': train_data['Qualification'].mode()[0],
    'Role': train_data['Role'].mode()[0],
    'Cert': train_data['Cert'].mode()[0]
}, inplace=True)

# Перевіримо, чи залишилися пропуски
missing_train_final = train_data.isnull().sum()
missing_train_final


Experience       0
Qualification    0
University       0
Role             0
Cert             0
Salary           0
Age              0
dtype: int64

### Крок 4: Масштабування числових ознак

In [7]:
# Крок 4: Масштабування числових ознак

# Категоризація змінної 'Experience'
def categorize_experience(years):
    if years < 3:
        return 'Junior'
    elif 3 <= years <= 7:
        return 'Mid'
    else:
        return 'Senior'

# Додаємо категорію до тренувального та валідаційного наборів
train_data['Experience_Category'] = train_data['Experience'].apply(categorize_experience)
valid_data['Experience_Category'] = valid_data['Experience'].apply(categorize_experience)

# Видаляємо числову ознаку 'Experience'
train_data.drop(columns=['Experience'], inplace=True)
valid_data.drop(columns=['Experience'], inplace=True)

# Масштабування лише числових змінних
numerical_features = ['Age']  # Лише 'Age', оскільки 'Experience' тепер є категоріальною

# Ініціалізація трансформерів
scaler_standard = StandardScaler()
scaler_power = PowerTransformer()

# Копіюємо дані для масштабування
train_data_standard = train_data.copy()
valid_data_standard = valid_data.copy()

train_data_power = train_data.copy()
valid_data_power = valid_data.copy()

# Масштабування за допомогою StandardScaler
train_data_standard[numerical_features] = scaler_standard.fit_transform(train_data[numerical_features])
valid_data_standard[numerical_features] = scaler_standard.transform(valid_data[numerical_features])

# Масштабування за допомогою PowerTransformer
train_data_power[numerical_features] = scaler_power.fit_transform(train_data[numerical_features])
valid_data_power[numerical_features] = scaler_power.transform(valid_data[numerical_features])

# Перевіримо результати
train_data_standard.head(), train_data_power.head()


(  Qualification University    Role Cert  Salary       Age Experience_Category
 0           Msc      Tier2     Mid  Yes   98000  1.346807                 Mid
 1           PhD      Tier2  Senior  Yes  135500 -2.079446                 Mid
 2           Msc      Tier2  Senior  Yes  123500 -1.160208                 Mid
 3           Bsc      Tier3     Mid   No   85000  1.513942                 Mid
 4           Bsc      Tier2  Senior  Yes  111500  1.513942                 Mid,
   Qualification University    Role Cert  Salary       Age Experience_Category
 0           Msc      Tier2     Mid  Yes   98000  1.407452                 Mid
 1           PhD      Tier2  Senior  Yes  135500 -1.945406                 Mid
 2           Msc      Tier2  Senior  Yes  123500 -1.172305                 Mid
 3           Bsc      Tier3     Mid   No   85000  1.597059                 Mid
 4           Bsc      Tier2  Senior  Yes  111500  1.597059                 Mid)

In [8]:
# Ініціалізація TargetEncoder
encoder = TargetEncoder()

# Визначення категоріальних і числових змінних
categorical_features = ['Qualification', 'University', 'Role', 'Cert', 'Experience_Category']
numerical_features = ['Age']

# Копіюємо дані для масштабування
train_data_encoded = train_data.copy()
valid_data_encoded = valid_data.copy()

# Кодуємо категоріальні змінні TargetEncoding
for col in categorical_features:
    train_data_encoded[col] = encoder.fit_transform(train_data[col], train_data['Salary'])
    valid_data_encoded[col] = encoder.transform(valid_data[col])

# Об'єднуємо числові та закодовані ознаки
all_features = numerical_features + categorical_features

# Масштабування всіх ознак
scaler_standard = StandardScaler()
scaler_power = PowerTransformer()

In [9]:
# Масштабування для StandardScaler
train_data_standard = train_data_encoded.copy()
valid_data_standard = valid_data_encoded.copy()

train_data_standard[all_features] = scaler_standard.fit_transform(train_data_encoded[all_features])
valid_data_standard[all_features] = scaler_standard.transform(valid_data_encoded[all_features])

# Масштабування для PowerTransformer
train_data_power = train_data_encoded.copy()
valid_data_power = valid_data_encoded.copy()

train_data_power[all_features] = scaler_power.fit_transform(train_data_encoded[all_features])
valid_data_power[all_features] = scaler_power.transform(valid_data_encoded[all_features])

In [10]:
# Розділення даних
X_train_standard = train_data_standard.drop(columns=['Salary'])
y_train_standard = train_data_standard['Salary']

X_valid_standard = valid_data_standard.drop(columns=['Salary'])
y_valid_standard = valid_data_standard['Salary']

X_train_power = train_data_power.drop(columns=['Salary'])
y_train_power = train_data_power['Salary']

X_valid_power = valid_data_power.drop(columns=['Salary'])
y_valid_power = valid_data_power['Salary']



In [12]:
# Оптимізація n_neighbors для KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

knn = KNeighborsRegressor()
param_grid = {'n_neighbors': range(1, 50)}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='neg_mean_absolute_percentage_error', cv=5, n_jobs=-1)

X_train = train_data_standard.drop(columns=['Salary'])
y_train = train_data_standard['Salary']

grid_search.fit(X_train, y_train)

best_n_neighbors = grid_search.best_params_['n_neighbors']
best_score = -grid_search.best_score_

print(f"Найкраще значення n_neighbors: {best_n_neighbors}")
print(f"MAPE для найкращого значення n_neighbors: {best_score:.2%}")

# Крок 5: Навчання моделі з оптимальним n_neighbors
best_knn_model = KNeighborsRegressor(n_neighbors=best_n_neighbors)
best_knn_model.fit(X_train, y_train)

# Підготовка валідаційного набору
X_valid = valid_data_standard.drop(columns=['Salary'])
y_valid = valid_data_standard['Salary']

# Продовження виконання моделі
y_pred = best_knn_model.predict(X_valid)
final_mape = mean_absolute_percentage_error(y_valid, y_pred)
final_mse = mean_squared_error(y_valid, y_pred)
final_r2 = r2_score(y_valid, y_pred)

print(f"Validation Results with Optimized n_neighbors:")
print(f" - Validation MAPE: {final_mape:.2%}")
print(f" - Validation MSE: {final_mse:.2f}")
print(f" - Validation R2: {final_r2:.2f}")

Найкраще значення n_neighbors: 3
MAPE для найкращого значення n_neighbors: 6.71%
Validation Results with Optimized n_neighbors:
 - Validation MAPE: 12.69%
 - Validation MSE: 171222857.14
 - Validation R2: 0.32
